Übung: Broker und Filter mit K-native
-----------------------------

![](https://user-images.githubusercontent.com/16281246/116248768-1fe56080-a73a-11eb-9a85-8bdccb82d16c.png)

Quelle: K-native

- - - 

**Broker**
Ein Broker in Knative ist eine zentrale Komponente, die als Verteiler von Ereignissen (Events) fungiert. Er empfängt Ereignisse von verschiedenen Quellen und sorgt dafür, dass diese an die richtigen Abonnenten weitergeleitet werden. Ein Broker besteht aus zwei Hauptteilen:

* Ingress: Hier kommen die Ereignisse an.
* Channel: Ein internes Kommunikationssystem, das die Ereignisse an die Abonnenten weiterleitet.

**Filter (Trigger)**
Ein Filter, auch Trigger genannt, definiert eine Regel, die bestimmt, welche Ereignisse an eine bestimmte Knative-Services oder Endpunkte gesendet werden. Triggers filtern die Ereignisse basierend auf bestimmten Kriterien, wie z.B. dem Ereignistyp oder anderen Attributen, und leiten sie dann an den entsprechenden Empfänger weiter.

**Zusammenspiel von Broker und Trigger**
* Ereignisaufnahme (Source): Der Broker empfängt Ereignisse von verschiedenen Quellen.
* Verteilung (Broker): Der Broker verteilt die Ereignisse an die entsprechenden Triggers.
* Filterung (Trigger): Triggers filtern die Ereignisse gemäß den definierten Regeln.
* Zustellung (Sink): Gefilterte Ereignisse werden an die entsprechenden Services oder Endpunkte zugestellt.

Durch diese Architektur ermöglicht Knative eine lose Kopplung von Ereignisquellen und -empfängern, was die Skalierbarkeit und Flexibilität von serverlosen Anwendungen erhöht.

- - -

Zuerst erstellen wir den Kubernetes Namespace



In [ ]:
import os
os.environ['NS_BRKR']='ms-brkr'
! kubectl create namespace ${NS_BRKR}
! # kubectl label  namespace ${NS_BRKR} istio-injection=enabled

### Dashboard

Jetzt ist ein guter Zeitpunkt um das Kubernetes Dashboard zu starten und dort im Pulldownmenu den Namespace "ms-brkr" auszuwählen.

Wählt nachfolgenden Link an und aktzeptiert das Zertifikat um dann ohne Token, drückt "Überspringen" oder "Skip", ins Dashboard zu wechseln.

In [ ]:
! echo "https://"$(cat ~/work/server-ip)":8443"

Anschliessend folgen die Standard Microservices

In [ ]:
%%bash
kubectl apply --namespace ${NS_BRKR} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/catalog-deployment.yaml
kubectl apply --namespace ${NS_BRKR} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/customer-deployment.yaml
kubectl apply --namespace ${NS_BRKR} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/order-deployment.yaml
kubectl apply --namespace ${NS_BRKR} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/webshop-deployment.yaml 
kubectl apply --namespace ${NS_BRKR} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/catalog-service.yaml
kubectl apply --namespace ${NS_BRKR} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/customer-service.yaml
kubectl apply --namespace ${NS_BRKR} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/order-service.yaml
kubectl apply --namespace ${NS_BRKR} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/webshop-service.yaml

In [ ]:
! echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ${NS_BRKR} webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop

**Shipment** und **Invoicing** starten wir mit genau einer Instanz.

**Sales** wird für diese Übung nicht benötigt und kann auch nicht angewählt werden.

In [ ]:
%%bash
kn service create kn-invoicing --scale 1 --image registry.gitlab.com/ch-mc-b/autoshop-ms/app/backoffice/invoicing:4.0.0 --port 8080 --namespace ${NS_BRKR}
kn service create kn-shipment  --scale 1 --image registry.gitlab.com/ch-mc-b/autoshop-ms/app/backoffice/shipment:4.0.0  --port 8080 --namespace ${NS_BRKR}

- - - 

### Hack

Wir erstellen pro K-native Service eine Kubernetes Service Object.

Dann sind die Microservices über das Menu ansprechbar.

In [ ]:
%%bash
cat <<EOF | kubectl apply --namespace ${NS_BRKR} -f - 
apiVersion: v1
kind: Service
metadata:
  name: shipment
spec:
  selector:
    serving.knative.dev/service: kn-shipment
  ports:
  - port: 8080
    protocol: TCP
    targetPort: 8080    
---
apiVersion: v1
kind: Service
metadata:
  name: invoicing
spec:
  selector:
    serving.knative.dev/service: kn-invoicing
  ports:
  - port: 8080
    protocol: TCP
    targetPort: 8080     
---      
apiVersion: v1
kind: Service
metadata:
  name: sales
spec:
  selector:
    serving.knative.dev/service: kn-sales
  ports:
  - port: 8080
    protocol: TCP
    targetPort: 8080     
EOF

- - -
### Broker

Für die Kommunkation erstellen wir einen Default Broker und stellen sicher, dass der Brocker (Ingress) mittels Port von aussen erreichbar ist.

In [ ]:
! kn broker create default --namespace ${NS_BRKR}
! kubectl patch service broker-ingress -n knative-eventing -p '{"spec": {"type": "LoadBalancer"}}'
! kubectl patch service broker-ingress -n knative-eventing --type='json' -p '[{"op":"replace","path":"/spec/ports/0/nodePort","value":30080}]'

- - -
### Trigger

Und drei Trigger welche gezielt die Nachrichten mit `type`: `shipment`, `invoicing` und `order` behandeln.

`order` wird an beide Microservices gesendent die anderen nur an die entsprechenden Namensvetter.

In [ ]:
%%bash
cat <<EOF | kubectl apply --namespace ${NS_BRKR} -f -
apiVersion: eventing.knative.dev/v1
kind: Trigger
metadata:
  name: invoicing
spec:
  broker: default
  filter:
    attributes:
      type: invoicing 
  subscriber:
    ref:
     apiVersion: serving.knative.dev/v1
     kind: Service
     name: kn-invoicing
EOF

In [ ]:
%%bash
cat <<EOF | kubectl apply --namespace ${NS_BRKR}  -f -
apiVersion: eventing.knative.dev/v1
kind: Trigger
metadata:
  name: shipment
spec:
  broker: default
  filter:
    attributes:
      type: shipment
  subscriber:
    ref:
     apiVersion: serving.knative.dev/v1
     kind: Service
     name: kn-shipment
EOF

In [ ]:
%%bash

cat <<EOF | kubectl apply --namespace ${NS_BRKR}  -f -
apiVersion: eventing.knative.dev/v1
kind: Trigger
metadata:
  name: invoicing-order
spec:
  broker: default
  filter:
    attributes:
      type: order 
  subscriber:
    ref:
     apiVersion: serving.knative.dev/v1
     kind: Service
     name: kn-invoicing
---
apiVersion: eventing.knative.dev/v1
kind: Trigger
metadata:
  name: shipment-order
spec:
  broker: default
  filter:
    attributes:
      type: order
  subscriber:
    ref:
     apiVersion: serving.knative.dev/v1
     kind: Service
     name: kn-shipment
EOF

### Gesamtübersicht

Die Subscriptions filtern `"Ce-Type:` und leiten die Events weiter an die Microservices **Shipment** und/oder **Invoicing**.

In [ ]:
%%bash
echo "Broker URL (intern)"
kubectl get broker default -o jsonpath='{.status.address.url}' --namespace ${NS_BRKR}
echo ""
echo "------------------------"
echo "Broker URL (extern)"
echo "http://$(cat ~/work/server-ip):"$(kubectl get svc broker-ingress -n knative-eventing -o jsonpath='{.spec.ports[?(@.port==80)].nodePort}')/${NS_BRKR}/default
echo "------------------------"
echo "Triggers"
kn trigger list --namespace ${NS_BRKR}
echo "------------------------"
echo "Microservice Menu"
echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ${NS_BRKR} webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop

- - -
### Testen

Zuerst senden wir einen Event an den Container **shipment**

In [ ]:
%%bash
PORT=$(kubectl get svc broker-ingress -n knative-eventing -o jsonpath='{.spec.ports[?(@.port==80)].nodePort}')
curl -s -X POST http://localhost:${PORT}/${NS_BRKR}/default \
-H "Host: broker-ingress.knative-eventing.svc.cluster.local" \
-H "Ce-Id: K-native-Broker" \
-H "Ce-Specversion: 1.0" \
-H "Ce-Type: shipment" \
-H "Ce-Source: curl" \
-H "Content-Type: application/json" \
-d '{ "product_id": 1, "quantity": 1, "customer_id": 1}'

Nun sollten wir eine weitere Bestellung haben

In [ ]:
! echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ${NS_BRKR} webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop/shipment/shipment

Dann senden wir einen Event an den Microservice **invoicing**

In [ ]:
%%bash
PORT=$(kubectl get svc broker-ingress -n knative-eventing -o jsonpath='{.spec.ports[?(@.port==80)].nodePort}')
curl -s -X POST http://localhost:${PORT}/${NS_BRKR}/default \
-H "Host: broker-ingress.knative-eventing.svc.cluster.local" \
-H "Ce-Id: K-native-Broker" \
-H "Ce-Specversion: 1.0" \
-H "Ce-Type: invoicing" \
-H "Ce-Source: curl" \
-H "Content-Type: application/json" \
-d '{ "product_id": 1, "quantity": 1, "customer_id": 2}'

echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ${NS_BRKR} webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop/invoicing/invoicing

Und zum Schluss an beide Microservices.

In [ ]:
%%bash
PORT=$(kubectl get svc broker-ingress -n knative-eventing -o jsonpath='{.spec.ports[?(@.port==80)].nodePort}')
curl -s -X POST http://localhost:${PORT}/${NS_BRKR}/default \
-H "Host: broker-ingress.knative-eventing.svc.cluster.local" \
-H "Ce-Id: K-native-Broker" \
-H "Ce-Specversion: 1.0" \
-H "Ce-Type: order" \
-H "Ce-Source: curl" \
-H "Content-Type: application/json" \
-d '{ "product_id": 1, "quantity": 1, "customer_id": 3}'

- - -

### Message Broker vs. K-native Eventing

In einer Microservices-Architektur können Services direkt mit einem Message Broker wie Kafka kommunizieren, indem sie Nachrichten senden und empfangen. Dies ermöglicht eine robuste und skalierbare Kommunikation, erfordert jedoch, dass die Microservices die spezifischen APIs und Konfigurationen des Brokers kennen und verwalten.

Mit K-native Eventing hingegen wird der Message Broker abstrahiert, wodurch die Microservices von den Details des Brokers entkoppelt werden. Knative Eventing bietet ein standardisiertes Eventing-Modell und erleichtert das Routing und Verwalten von Events. Dies ermöglicht eine flexiblere und einfacher zu verwaltende Architektur, da die Microservices sich auf das Verarbeiten von Events konzentrieren können, ohne sich um die Details der Broker-Implementierung kümmern zu müssen.

Hier der gekürzte Code 

- - -

    @app.route('/', methods=['POST'])
    def add_order():
        try:
            data = request.get_json()
            if not data or not all(key in data for key in ("product_id", "quantity", "customer_id")):
                return jsonify({"status": "error", "message": "Invalid request"}), 400

            new_order_id = max(order['id'] for order in orders) + 1 if orders else 1
            new_order = {
                "id": new_order_id,
                "product_id": data["product_id"],
                "quantity": data["quantity"],
                "customer_id": data["customer_id"]
            }
            orders.append(new_order)
            return jsonify({"status": "success", "data": new_order}), 201


- - -

Aufräumen


In [ ]:
! kn service delete kn-shipment  --namespace ${NS_BRKR}
! kn service delete kn-invoicing --namespace ${NS_BRKR}
! kn service delete kn-sales --namespace ${NS_BRKR}
! kubectl delete namespace ${NS_BRKR}

- - -
### Quellen

* Sourcecode: https://gitlab.com/ch-mc-b/autoshop-ms/app
* Kubernetes Deklarationen: https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates
* Container Registry: https://gitlab.com/ch-mc-b/autoshop-ms/app/shop/container_registry